###   Search Engine with tools and agents

In [1]:
##  Arxiv  -- > research
##  tools creation

from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [2]:
## Used the inbuilt tool of Wikipedia
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=250)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki)

# Now you can use the wikipedia_tool
# For example:
wiki.name

'wikipedia'

In [3]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)

# Now you can use the wikipedia_tool
# For example:
arxiv.name

'arxiv'

In [4]:
tools = [wiki,arxiv]

In [5]:

## Custom tools [RA6 Tool]
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.tools import tool
from typing import Optional, List
import os


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
# Load web content
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents, OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [7]:
# First, let's check what's actually available in langchain.tools
import langchain.tools

# List all available attributes in langchain.tools
print("Available in langchain.tools:")
print(dir(langchain.tools))

# Check if there's a retriever module
print("\n\nChecking for retriever module...")
try:
    import langchain.tools.retriever
    print("langchain.tools.retriever exists")
    print("Available in retriever:", dir(langchain.tools.retriever))
except ImportError as e:
    print(f"langchain.tools.retriever doesn't exist: {e}")

# Check your langchain version
import langchain
print(f"\n\nLangChain version: {langchain.__version__}")

Available in langchain.tools:
['BaseTool', 'InjectedState', 'InjectedStore', 'InjectedToolArg', 'InjectedToolCallId', 'ToolException', 'ToolRuntime', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'tool', 'tool_node']


Checking for retriever module...
langchain.tools.retriever doesn't exist: No module named 'langchain.tools.retriever'


LangChain version: 1.2.6


In [8]:
!pip install -U langchain langchain-community langchain-openai


  Attempting uninstall: langgraph-sdk

    Found existing installation: langgraph-sdk 0.2.10

    Uninstalling langgraph-sdk-0.2.10:

      Successfully uninstalled langgraph-sdk-0.2.10

   ---------------------------------------- 0/4 [langgraph-sdk]
   ---------------------------------------- 0/4 [langgraph-sdk]
   ---------------------------------------- 0/4 [langgraph-sdk]
  Attempting uninstall: langgraph-prebuilt
   ---------------------------------------- 0/4 [langgraph-sdk]
    Found existing installation: langgraph-prebuilt 1.0.5
   ---------------------------------------- 0/4 [langgraph-sdk]
    Uninstalling langgraph-prebuilt-1.0.5:
   ---------------------------------------- 0/4 [langgraph-sdk]
      Successfully uninstalled langgraph-prebuilt-1.0.5
   ---------------------------------------- 0/4 [langgraph-sdk]
   ---------- ----------------------------- 1/4 [langgraph-prebuilt]
   ---------- ----------------------------- 1/4 [langgraph-prebuilt]
  Attempting uninstall: la

In [9]:
from langchain_core.tools.retriever import create_retriever_tool
# This creates a tool that an agent can use to look up information from your vector store
retriever_tool = create_retriever_tool(
    retriever, 
    "langsmith-search", 
    "Search any information about Langsmith"
)

# This returns the name of the tool
retriever_tool.name

'langsmith-search'

In [10]:
tools = [wiki, arxiv, retriever_tool]

In [11]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\mahad\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 StructuredTool(name='langsmith-search', description='Search any information about Langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=<function create_retriever_tool.<locals>.func at 0x00000170F0E1C4A0>, coroutine=<function create_retriever_tool.<locals>.afunc at 0x00000170F0E1C5E0>)]

In [13]:
## Run all this tools with Agents and LLM Models

## Tools, LLM-->AgentExecutor
from langchain_groq import ChatGroq
import openai
from dotenv import load_dotenv
load_dotenv()
import os

# You need these imports to fix the squiggly lines in your image
import openai 
from langchain_groq import ChatGroq

groq_api_key = os.getenv("GROQ_API_KEY")
openai.api_key = os.getenv("OPENAI_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key, model_name="Llama3-8b-8192")

In [33]:
!pip install langchainhub
!pip install -U langchain langchain-community langchain-openai langchainhub


In [35]:
from langchain import hub

prompt = hub.pull("hwchase1/openai-functions-agent")

for i, message in enumerate(prompt.messages):
    print(f"[{i}] {type(message).__name__}")
    print(f"   {message}")


ImportError: cannot import name 'hub' from 'langchain' (c:\Users\mahad\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\__init__.py)

In [30]:
from langchain.agents import initialize_agent, AgentType
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True
)


ImportError: cannot import name 'initialize_agent' from 'langchain.agents' (c:\Users\mahad\AppData\Local\Programs\Python\Python312\Lib\site-packages\langchain\agents\__init__.py)